<a href="https://colab.research.google.com/github/Farmhouse121/Adventures-in-Financial-Data-Science/blob/main/Book/Section%202.3%20The%20U.S.%20Stock%20Market%20Through%20Time/Section%202.3.1%20Drift%20and%20Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Installing yfinance and arch and getting the data...")
!pip install arch 1>/dev/null
!pip install yfinance 1>/dev/null
from yfinance import download
import pandas as pd
import numpy as np ;
import matplotlib.pyplot as pl
from datetime import datetime
zero,one,two,three,five,ten,hundred=0e0,1e0,2e0,3e0,5e0,1e1,1e2 # some friendly numbers
half,GoldenRatio=one/two,(one+np.sqrt(five))/two

# get the daily returns of the S&P 500 
SPX=download('^GSPC','1928-01-03').dropna()
SPX['Return']=SPX['Adj Close'].pct_change()*hundred
SPX.index=pd.DatetimeIndex(SPX.index).to_period('B')
SPX.dropna(inplace=True)
SPX.loc[SPX["Volume"]==0,"Volume"]=np.nan
SPX

Installing yfinance and arch and getting the data...
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Return
Date,,,,,,,
1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,NaN,-0.225230
1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,NaN,-0.959368
1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,NaN,0.626784
1928-01-09,17.500000,17.500000,17.500000,17.500000,17.500000,NaN,-0.906001
1928-01-10,17.370001,17.370001,17.370001,17.370001,17.370001,NaN,-0.742852
...,...,...,...,...,...,...,...
2023-05-18,4157.680176,4202.200195,4153.500000,4198.049805,4198.049805,3.980500e+09,0.944505
2023-05-19,4204.149902,4212.910156,4180.200195,4191.979980,4191.979980,4.041900e+09,-0.144587
2023-05-22,4190.779785,4209.220215,4179.680176,4192.629883,4192.629883,3.728520e+09,0.015503


In [2]:
# get annual series of proportion of year in recession
from pandas_datareader.data import DataReader
USREC=DataReader("USREC","fred",SPX.index[0].to_timestamp(),SPX.index[-1].to_timestamp()).rename(columns={"USREC":"Recession"})
USREC.index=pd.DatetimeIndex(USREC.index).to_period("D")
USREC["year"]=list(map(lambda x:x.year,USREC.index))
USREC=USREC.groupby("year").mean()
USREC

,Recession
year,
1928,0.000000
1929,0.333333
1930,1.000000
1931,1.000000
1932,1.000000
...,...
2019,0.000000
2020,0.166667
2021,0.000000


In [6]:
# fit a GARCH model using arch package
print("Installing ARCH and fitting model with GED innovations.")
from arch.univariate import ARX, GARCH, GeneralizedError

class GeneralizedError2(GeneralizedError):
    """Version of the GARCH process modified to change the bounds so that Laplace models etc. can be fitted."""
    def bounds(self,*args):
        """Revised bounds."""
        return [(0e0,1e2)]
    
    def constraints(self,*args):
        return np.array([[1], [-1]]), np.array([self.bounds()[0][0],-self.bounds()[0][1]])

from tqdm.notebook import tqdm
results=pd.DataFrame({"year":[],"mu":[],"phi":[],"C":[],"A":[],"B":[],"kappa":[]}).set_index("year")
results.index=pd.DatetimeIndex(results.index).to_period('Y')

for year in tqdm(range(SPX.index[0].year,SPX.index[-1].year+1)):
    model = ARX(SPX.loc[SPX.index.year==year,"Return"],lags=[1])
    model.volatility = GARCH(1, 0, 1)
    model.distribution = GeneralizedError()
    fit=model.fit(update_freq=0,disp='off')

    if not fit.convergence_flag:
        mu,phi,C,A,B,nu=tuple(fit.params)
        results=pd.concat([results,pd.DataFrame({"year":[year],"mu":[mu],"phi":[phi],"C":[C],"A":[A],"B":[B],"kappa":[nu/two]}).set_index("year")])

results

Installing ARCH and fitting model with GED innovations.


  0%|          | 0/96 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/arch/univariate/distribution.py:1194: RuntimeWarning: overflow encountered in power
  lls -= 0.5 * abs(resids / (sqrt(sigma2) * c)) ** nu
/usr/local/lib/python3.10/dist-packages/arch/univariate/distribution.py:1194: RuntimeWarning: overflow encountered in power
  lls -= 0.5 * abs(resids / (sqrt(sigma2) * c)) ** nu
/usr/local/lib/python3.10/dist-packages/arch/univariate/distribution.py:1194: RuntimeWarning: overflow encountered in power
  lls -= 0.5 * abs(resids / (sqrt(sigma2) * c)) ** nu


,mu,phi,C,A,B,kappa
year,,,,,,
1928,0.213503,0.045765,1.285717e-01,0.101987,0.750586,0.674925
1929,0.216339,-0.053070,1.746877e-01,0.249382,0.724788,0.569474
1930,0.112767,-0.106637,1.268134e-01,0.263272,0.730457,0.836508
1931,-0.326928,-0.132279,3.187037e-01,0.097781,0.852799,0.664308
1932,-0.126065,0.023032,1.451915e+00,0.112960,0.757731,0.795575
...,...,...,...,...,...,...
2019,0.134195,-0.054537,4.121622e-02,0.217338,0.717367,0.672854
2020,0.251211,-0.217351,8.888464e-02,0.301122,0.698878,0.632700
2021,0.131095,-0.022648,1.460237e-01,0.284804,0.520630,0.681681


In [7]:
# Figure 2.10
figure,plot=pl.subplots(figsize=(ten*GoldenRatio,ten))
figure.suptitle("\nEstimated Annual Average Daily Returns for S&P 500",fontsize=22)
plot.set_title("%s to %s" % (min(SPX.index),max(SPX.index)),fontsize=18)
positive,negative=results["mu"]>zero,results["mu"]<zero
plot.bar(results.index[positive],results.loc[positive,"mu"],color='blue',label="$\hat\mu$")
plot.bar(results.index[negative],results.loc[negative,"mu"],color='red',label="$\hat\mu$")
plot.axhline(color='black',lw=1)

for year in [y for y in USREC.index if USREC.loc[y,"Recession"]]:
    plot.axvspan(year-half,year+half,facecolor="black",alpha=0.1)

plot.set_ylabel("Daily Return (%)",fontsize=14)
plot.axhline(results["mu"].mean(),color='black',lw=1);

In [8]:
# Figure 2.11
figure,plot=pl.subplots(figsize=(ten*GoldenRatio,ten))
figure.suptitle("\nEstimated Autocorrelation of Daily Returns for S&P 500",fontsize=22)
plot.set_title("%s to %s" % (min(SPX.index),max(SPX.index)),fontsize=18)
plot.plot(results.index,results["phi"],'-',color="blue",label="$\\hat{\\phi}$")

plot.axhline(color='black',lw=1)
fisher_error=one/(np.sqrt(252)-three)
plot.axhspan(-fisher_error,fisher_error,color='black',alpha=0.2)
plot.axhspan(-two*fisher_error,two*fisher_error,color='black',alpha=0.2)
#plot.axhline(-two*fisher_error,color='black',linestyle=':',linewidth=1)
#plot.axhline(+two*fisher_error,color='black',linestyle=':',linewidth=1)

plot.set_ylabel("1st. Lag Correlation",fontsize=14);